In [32]:
import polars as pl

df = pl.read_csv('crsp_a_stock_10yrs.csv', schema_overrides=pl.Schema({'date': pl.Date}))
df

PERMNO,date,TICKER,PRC
i64,date,str,f64
10001,2013-12-30,"""EGAS""",8.12
10001,2013-12-31,"""EGAS""",8.03
10001,2014-01-02,"""EGAS""",8.04
10001,2014-01-03,"""EGAS""",8.18
10001,2014-01-06,"""EGAS""",8.34
…,…,…,…
93436,2023-12-22,"""TSLA""",252.53999
93436,2023-12-26,"""TSLA""",256.60999
93436,2023-12-27,"""TSLA""",261.44


In [40]:
from tqdm import tqdm
import pandas as pd

tickers = sorted(df['TICKER'].drop_nulls().unique().to_list())

new_df = []

for date, group in tqdm(df.group_by(['date'], maintain_order=True), total=df['date'].n_unique()):
    row = [date[0]]
    for ticker in tickers:
        results = group.filter(pl.col('TICKER') == ticker)
        if len(results) == 1:
            row.append(results['PRC'].to_list()[0])
        else:
            # Problem with conflicting ticker name if len(results) > 1
            # missing data if results is len(results) == 0
            row.append(float('nan'))
    new_df.append(row)

new_df = pd.DataFrame(new_df, columns=['date'] + tickers)
new_df = new_df.set_index('date').sort_index()

100%|██████████| 2525/2525 [8:37:03<00:00, 12.29s/it]     


In [41]:
new_df

,A,AA,AAA,AAAP,AAAU,AABA,AAC,AACG,AACI,AACQ,...,ZVRA,ZVSA,ZWRK,ZWS,ZX,ZY,ZYME,ZYNE,ZYXI,ZZZ
date,,,,,,,,,,,,,,,,,,,,,
2013-12-30,57.32000,10.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.47,NaN,NaN,NaN,NaN,NaN
2013-12-31,57.19000,10.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,NaN
2014-01-02,56.21000,10.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,NaN
2014-01-03,56.92000,10.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.51,NaN,NaN,NaN,NaN,NaN
2014-01-06,56.64000,10.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.65,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,139.57001,33.77,25.055,NaN,20.335,NaN,NaN,0.8130,-10.755,NaN,...,4.97,0.8198,NaN,29.94,NaN,NaN,9.83,NaN,10.64,NaN
2023-12-26,139.81000,33.87,25.045,NaN,20.480,NaN,NaN,0.8391,-10.755,NaN,...,4.95,0.8110,NaN,30.16,NaN,NaN,10.68,NaN,10.71,NaN
2023-12-27,139.82001,34.81,25.120,NaN,20.585,NaN,NaN,1.0500,-10.750,NaN,...,5.48,0.8500,NaN,29.93,NaN,NaN,10.75,NaN,10.77,NaN


In [42]:
new_df.to_csv('crsp.csv')